In [36]:
test_input = """R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)
"""

with open("inputs/d18") as f:
    input = f.read()

In [54]:
def parse_input(input):
    plan = [
        dict(zip(["direction", "meters", "color"], line.split()))
        for line in input.strip().splitlines()
    ]

    for step in plan:
        step["meters"] = int(step["meters"])
        step["color"] = step["color"].strip("()")

    return plan

In [58]:
test_plan = parse_input(test_input)
test_plan

[{'direction': 'R', 'meters': 6, 'color': '#70c710'},
 {'direction': 'D', 'meters': 5, 'color': '#0dc571'},
 {'direction': 'L', 'meters': 2, 'color': '#5713f0'},
 {'direction': 'D', 'meters': 2, 'color': '#d2c081'},
 {'direction': 'R', 'meters': 2, 'color': '#59c680'},
 {'direction': 'D', 'meters': 2, 'color': '#411b91'},
 {'direction': 'L', 'meters': 5, 'color': '#8ceee2'},
 {'direction': 'U', 'meters': 2, 'color': '#caa173'},
 {'direction': 'L', 'meters': 1, 'color': '#1b58a2'},
 {'direction': 'U', 'meters': 2, 'color': '#caa171'},
 {'direction': 'R', 'meters': 2, 'color': '#7807d2'},
 {'direction': 'U', 'meters': 3, 'color': '#a77fa3'},
 {'direction': 'L', 'meters': 2, 'color': '#015232'},
 {'direction': 'U', 'meters': 2, 'color': '#7a21e3'}]

In [57]:
plan = parse_input(input)

In [55]:
def build_boundary(plan):
    boundary = set()
    min_row, min_col, max_row, max_col, row, col = 0, 0, 0, 0, 0, 0
    direction_moves = {"R": (0, 1), "L": (0, -1), "U": (-1, 0), "D": (1, 0)}

    for step in plan:
        move = direction_moves[step["direction"]]
        for _ in range(step["meters"]):
            row = row + move[0]
            col = col + move[1]
            boundary.add((row, col))
        min_row = min(min_row, row)
        min_col = min(min_col, col)
        max_row = max(max_row, row)
        max_col = max(max_col, col)

    return {"boundary": boundary, "min_row": min_row, "max_row": max_row, "min_col": min_col, "max_col": max_col}

In [65]:
%%time

test_boundary = build_boundary(test_plan)
len(test_boundary["boundary"]) == 38

CPU times: user 0 ns, sys: 41 µs, total: 41 µs
Wall time: 43.6 µs


True

In [67]:
%%time

boundary = build_boundary(plan)
len(boundary["boundary"])

CPU times: user 1.27 ms, sys: 0 ns, total: 1.27 ms
Wall time: 1.27 ms


3650

In [100]:
def visualize(boundary: dict, inner: set):
    for row in range(boundary["min_row"], boundary["max_row"] + 1):
        for col in range(boundary["min_col"], boundary["max_col"] + 1):
            if (row, col) in boundary["boundary"]:
                print("#", end="")
            elif (row, col) in inner:
                print("x", end="")
            else:
                print(".", end="")
        print()

In [96]:
visualize(test_boundary, set())

#######
#.....#
###...#
..#...#
..#...#
###.###
#...#..
##..###
.#....#
.######


In [84]:
?list.pop

Signature: list.pop(self, index=-1, /)
Docstring:
Remove and return item at index (default last).

Raises IndexError if list is empty or index is out of range.
Type:      method_descriptor

In [109]:
def flood_fill(boundary: dict):
    def find_start():
        """Find left-most point we can start flood fill from"""
        for row in range(boundary["min_row"], boundary["max_row"] + 1):
            if (row, 0) in boundary["boundary"]:
                return (row, 0)

    (row, col) = find_start()
    row += 1
    col += 1

    neighbours = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    frontier = [(row, col)]
    found = set()
    while frontier:
        (row, col) = frontier.pop()
        for (delta_row, delta_col) in neighbours:
            neighbour = (row + delta_row, col + delta_col)
            if neighbour[0] >= boundary["min_row"] and neighbour[0] <= boundary["max_row"] and \
                neighbour[1] >= boundary["min_col"] and neighbour[1] <= boundary["max_col"] and \
                neighbour not in boundary["boundary"] and neighbour not in found:

                found.add(neighbour)
                frontier.append(neighbour)
    return found


In [110]:
test_inner = flood_fill(test_boundary)
visualize(test_boundary, test_inner)

#######
#xxxxx#
###xxx#
..#xxx#
..#xxx#
###x###
#xxx#..
##xx###
.#xxxx#
.######


In [111]:
len(test_inner) + len(test_boundary["boundary"]) == 62

True

In [112]:
inner = flood_fill(boundary)
visualize(boundary, inner)

..........................................................................................................................................................................................................................................................................####....................................................................................................................................
..........................................................................................................................................................................................................................................................................#xx#....................................................................................................................................
..................................................................................................................................................................................................

In [107]:
def p1(plan):
    boundary = build_boundary(plan)
    inner = flood_fill(boundary)
    return len(boundary["boundary"]) + len(inner)

In [106]:
len(inner) + len(boundary["boundary"]) == 56923

56923

In [115]:
test_plan

[{'direction': 'R', 'meters': 6, 'color': '#70c710'},
 {'direction': 'D', 'meters': 5, 'color': '#0dc571'},
 {'direction': 'L', 'meters': 2, 'color': '#5713f0'},
 {'direction': 'D', 'meters': 2, 'color': '#d2c081'},
 {'direction': 'R', 'meters': 2, 'color': '#59c680'},
 {'direction': 'D', 'meters': 2, 'color': '#411b91'},
 {'direction': 'L', 'meters': 5, 'color': '#8ceee2'},
 {'direction': 'U', 'meters': 2, 'color': '#caa173'},
 {'direction': 'L', 'meters': 1, 'color': '#1b58a2'},
 {'direction': 'U', 'meters': 2, 'color': '#caa171'},
 {'direction': 'R', 'meters': 2, 'color': '#7807d2'},
 {'direction': 'U', 'meters': 3, 'color': '#a77fa3'},
 {'direction': 'L', 'meters': 2, 'color': '#015232'},
 {'direction': 'U', 'meters': 2, 'color': '#7a21e3'}]

In [ ]:
?str.removeprefix

In [122]:
def reinterpret(plan):
    directions = ["R", "D", "L", "U"]
    steps = []

    for step in plan:
        digits = step["color"].removeprefix("#")
        meters = int(digits[0:5], 16)
        direction = directions[int(digits[5])]

        steps.append({"direction": direction, "meters": meters})

    return steps

In [123]:
reinterpret(test_plan)

[{'direction': 'R', 'meters': 461937},
 {'direction': 'D', 'meters': 56407},
 {'direction': 'R', 'meters': 356671},
 {'direction': 'D', 'meters': 863240},
 {'direction': 'R', 'meters': 367720},
 {'direction': 'D', 'meters': 266681},
 {'direction': 'L', 'meters': 577262},
 {'direction': 'U', 'meters': 829975},
 {'direction': 'L', 'meters': 112010},
 {'direction': 'D', 'meters': 829975},
 {'direction': 'L', 'meters': 491645},
 {'direction': 'U', 'meters': 686074},
 {'direction': 'L', 'meters': 5411},
 {'direction': 'U', 'meters': 500254}]

In [ ]:
def p2(plan):
    real_plan = reinterpret(plan)

    vertices = []
    min_row, min_col, max_row, max_col, row, col = 0, 0, 0, 0, 0, 0
    direction_moves = {"R": (0, 1), "L": (0, -1), "U": (-1, 0), "D": (1, 0)}

    for step in plan:
        move = direction_moves[step["direction"]]
        for _ in range(step["meters"]):
            row = row + move[0]
            col = col + move[1]
            boundary.add((row, col))
        min_row = min(min_row, row)
        min_col = min(min_col, col)
        max_row = max(max_row, row)
        max_col = max(max_col, col)

    return {"boundary": boundary, "min_row": min_row, "max_row": max_row, "min_col": min_col, "max_col": max_col}